# Examples highlighting eQuilibrator use on arbitrary compounds
This notebook gives examples of how to use the modified equilibrator program with custom ccache

In [1]:
import equilibrator_a as ea
from equilibrator_cache import Q_
from component_contribution.predict import GibbsEnergyPredictor
GP = GibbsEnergyPredictor()

# if the ./cache/compounds.sqlite does not exist, importing equilibrator_a will generate this from equilibrator/cache
# see next section for specific quilt repos

Fragments already downloaded
Local compounds.sqlite not found. Exporting from default equilibrator/cache quilt repo.
If another quilt repo is to be used, run generate_new_ccache with specified repo.
Fragments already downloaded
Exporting file 2 of 2 [00:02]: compounds.sqlite
Fragments already downloaded


# Resetting ccache
In the event the ccache (.sqlite file) should be reset. equilibrator_a will automatically check for a .sqlite file and if it doesn't exist generate it from the default equilibrator/cache quitl repo. A specific quilt repo can be specified as well.

In [2]:
# resetting the cache to a specified quilt repo
ea.generate_new_ccache('equilibrator/cache')

Attempting to install equilibrator/cache from quilt.
Fragments already downloaded
Removing old cache and generate new one
Exporting file 2 of 2 [00:02]: compounds.sqlite


# Generating compound objects and saving to the .sqlite

In [3]:
# get_compound first tries to find a compound in the ccache that matches the smiles. If not found it calculates
# using cxcalc. Both functions return a compound object
mol_string = 'CCO'
cpd_get = ea.get_compound(mol_string)

# A compound with an id less than ~51000 is already existing in the database
print(f'cpd_get id: {cpd_get.id}')

cpd_get id: 287


In [5]:
# For a compound not in ccache cpd_get and cpd_get inserts into the ccache and assigns an id in the fifty-thousands range. 
# Using an arbitrary compound here.
mol_string = 'C(O)CCCCCCCO'
cpd_get = ea.get_compound(mol_string)
cpd_id = cpd_get.id
# The new compound has a large id
print(f'cpd_get id: {cpd_id}')

cpd_get id: 51439


# Calculating ∆Go and ∆G'o 
This works the exact same way as equilibrator code

In [6]:
# Calculating ∆Go
# Comparing results from the two methods for a compound in ccache and NOT in the training data
mol_smiles = 'C(=O)CCCC(O)=O'
cpd_get = ea.get_compound(mol_smiles)
print(f'cpd_get id: {cpd_get.id}')

print(GP.standard_dgf(cpd_get))

cpd_get id: 1280
(-453.0 +/- 1.3) kilojoule / mole


In [7]:
# Calculating ∆G'o for a compound in the ccache without magensium data and not in the training data
mol_smiles = 'C(=O)CCOCCOCC(O)=O'
cpd_get = ea.get_compound(mol_smiles)
cond = {
    'p_h': Q_(7),
    'ionic_strength': Q_('0.1M'),
    'temperature': Q_('298.15K'),
    'p_mg': Q_(3)}
    
print(f'cpd_get id: {cpd_get.id}')
print(GP.standard_dgf_prime(cpd_get, **cond))

cpd_get id: 694325
(-229.9 +/- 3.4) kilojoule / mole
